In [6]:
! ls | grep -i png

edges.png
modelo.png
out.png


In [2]:
# ! conda remove opencv
# ! conda install -c menpo opencv
# ! pip install --upgrade pip
# ! pip install opencv-contrib-python

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
factor = 3

In [15]:
img = cv2.imread('modelo.png',)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
height, width = img.shape
n_height, n_width = int(width/factor), int(height/factor)
img = cv2.resize(img, (n_height, n_width))
# img = cv2.medianBlur(img, 1)

In [22]:
# plt.figure(figsize=(10, 12))
# plt.imshow(img, aspect="auto")
cv2.imshow("modelo", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
5*120

600

In [99]:
circles = cv2.HoughCircles(
    image=img, 
    method=cv2.HOUGH_GRADIENT, 
    dp=1, 
    minDist=img.shape[0]/64, 
    param1=200, 
    param2=10,
    minRadius=int(15/factor), 
    maxRadius=int(15/factor)
)
print(circles.shape)
img.shape[0]

(1, 609, 3)


733

In [100]:
img_colored = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        cv2.circle(img_colored, (i[0], i[1]), i[2], (0, 255, 0), cv2.FILLED)

In [101]:
cv2.imshow("modelo", img_colored)
cv2.waitKey(0)
cv2.destroyAllWindows()